<a href="https://colab.research.google.com/github/BaikaiL/voice/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 安装 Edge-TTS (用于高质量语音合成)
!pip install edge-tts nest_asyncio

In [1]:
!pip install pyecharts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 4.6 MB/s eta 0:00:00


In [2]:
!pip install plotly

In [1]:
# 1. 确保安装必要的库
!pip install psutil matplotlib

In [5]:
# 下载中文字体 (SimHei) 解决 Matplotlib 乱码问题
!wget -O SimHei.ttf "https://github.com/StellarCN/scp_zh/raw/master/fonts/SimHei.ttf"

--2026-01-12 02:42:59--  https://github.com/StellarCN/scp_zh/raw/master/fonts/SimHei.ttf
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/StellarCN/scp_zh/master/fonts/SimHei.ttf [following]
--2026-01-12 02:43:00--  https://raw.githubusercontent.com/StellarCN/scp_zh/master/fonts/SimHei.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9751960 (9.3M) [application/octet-stream]
Saving to: ‘SimHei.ttf’

SimHei.ttf          100%[===================>]   9.30M  33.9MB/s    in 0.3s    

2026-01-12 02:43:01 (33.9 MB/s) - ‘SimHei.ttf’ saved [9751960/9751960]



In [7]:
import matplotlib.font_manager as fm
# 注册中文字体
my_font = fm.FontProperties(fname="SimHei.ttf")
# 备用方案：尝试设置全局字体，防止某些未手动指定的地方乱码
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import gradio as gr
import torch
import asyncio
import nest_asyncio
import edge_tts
from transformers import pipeline, AutoProcessor, AutoModelForSpeechSeq2Seq
import os
import uuid
import psutil
import matplotlib.pyplot as plt
from collections import deque
import time

In [8]:
# 解决异步问题
nest_asyncio.apply()

# 监控工具类
class SystemMonitor:
    def __init__(self, history_len=60):
        self.history_len = history_len
        self.ram_usage = deque(maxlen=history_len)
        self.vram_usage = deque(maxlen=history_len)
        self.start_time = time.time()

    def get_plot(self):
        """获取当前系统状态并生成图表"""
        # 获取数据
        ram_percent = psutil.virtual_memory().percent
        self.ram_usage.append(ram_percent)

        if torch.cuda.is_available():
            vram_gb = torch.cuda.memory_allocated() / (1024**3)
        else:
            vram_gb = 0
        self.vram_usage.append(vram_gb)

        # 绘图
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3), dpi=100)
        ax1.plot(list(self.ram_usage), color='#ff7f0e', linewidth=2)

        # 使用 fontproperties 指定中文字体
        ax1.set_title(f"系统内存 (RAM): {ram_percent}%", fontsize=11, fontweight='bold', fontproperties=my_font)
        ax1.set_ylabel("使用率 (%)", fontproperties=my_font)
        ax1.set_ylim(0, 100)
        ax1.grid(True, linestyle='--', alpha=0.6)
        ax1.fill_between(range(len(self.ram_usage)), list(self.ram_usage), color='#ff7f0e', alpha=0.2)

        # VRAM 图表
        current_vram = self.vram_usage[-1] if self.vram_usage else 0
        total_vram = torch.cuda.get_device_properties(0).total_memory / (1024**3) if torch.cuda.is_available() else 16

        ax2.plot(list(self.vram_usage), color='#1f77b4', linewidth=2)
        ax2.set_title(f"GPU 显存占用: {current_vram:.2f} GB / {total_vram:.1f} GB", fontsize=11, fontweight='bold', fontproperties=my_font)
        ax2.set_ylabel("显存 (GB)", fontproperties=my_font)
        ax2.set_ylim(0, total_vram)
        ax2.grid(True, linestyle='--', alpha=0.6)
        ax2.fill_between(range(len(self.vram_usage)), list(self.vram_usage), color='#1f77b4', alpha=0.2)

        plt.tight_layout()
        plt.close()
        return fig

monitor = SystemMonitor()

In [ ]:
# 2. 后端逻辑
class CantonesToMandarinTranslator:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

        print(f"初始化模型到 {self.device}...")

        # ASR
        processor = AutoProcessor.from_pretrained("openai/whisper-small")
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            "baikai1022/whisper-small-cantonese-v2",
            torch_dtype=self.torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True
        ).to(self.device)

        self.asr_pipe = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            device=self.device,
            torch_dtype=self.torch_dtype,
        )

        # MT
        self.mt_pipe = pipeline(
            "translation",
            model="baikai1022/nllb-cantonese-mandarin-v1",
            device=self.device,
            torch_dtype=self.torch_dtype,
            src_lang="yue_Hant",
            tgt_lang="zho_Hans"
        )
        print("模型加载完毕！")

    def process(self, audio_path):
        if not audio_path:
            return "请先录音或上传文件", "", None

        print(f"正在处理: {audio_path}")

        # ASR
        asr_result = self.asr_pipe(audio_path, generate_kwargs={"language": "chinese"})
        cantonese_text = asr_result["text"]

        # MT
        mt_result = self.mt_pipe(cantonese_text)
        mandarin_text = mt_result[0]['translation_text']

        # TTS
        output_filename = f"out_{uuid.uuid4().hex[:8]}.mp3"
        voice = "zh-CN-XiaoxiaoNeural"
        async def _gen_tts():
            comm = edge_tts.Communicate(mandarin_text, voice)
            await comm.save(output_filename)
        asyncio.run(_gen_tts())

        return cantonese_text, mandarin_text, output_filename

# 实例化模型
if 'translator' not in globals():
    translator = CantonesToMandarinTranslator()

In [9]:
# 3. 前端界面
def gradio_process(audio):
    return translator.process(audio)

def update_monitor():
    return monitor.get_plot()

with gr.Blocks(title="粤语转普通话 AI") as demo:
    gr.Markdown("# 🇭🇰 粤语 -> 🇨🇳 普通话 智能转换器")

    # --- 上半部分：翻译 ---
    with gr.Row(variant="panel"): # 加个 panel 样式稍微区分一下
        # 左边输入
        with gr.Column(scale=1):
            input_audio = gr.Audio(sources=["microphone", "upload"], type="filepath", label="请说粤语")
            btn = gr.Button("开始转换", variant="primary", size="lg")

        # 右边输出
        with gr.Column(scale=1):
            txt_cantonese = gr.Textbox(label="识别结果 (粤语)", interactive=False)
            txt_mandarin = gr.Textbox(label="翻译结果 (普通话)", interactive=False)
            out_audio = gr.Audio(label="合成语音 (普通话)", autoplay=True)

    # --- 下半部分：监控区 ---
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 系统实时监控")
            monitor_plot = gr.Plot(label="硬件资源占用", show_label=False)

            # 定时器：每1秒刷新一次
            timer = gr.Timer(value=1)
            timer.tick(fn=update_monitor, outputs=monitor_plot)

    # 绑定事件
    btn.click(
        fn=gradio_process,
        inputs=input_audio,
        outputs=[txt_cantonese, txt_mandarin, out_audio]
    )

print("正在启动 Gradio...")
demo.launch(share=True, debug=True)

正在启动 Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8187b4479e576a9633.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎙️ 正在处理: /tmp/gradio/6eff450a880aa1b636ff62ea458b13b28f7c803672acb57926e9047e4bfaf2ed/audio.wav


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8187b4479e576a9633.gradio.live
